In [1]:
import os
import numpy as np
import pandas as pd
from time import process_time
from glob import glob
from multiprocessing import Pool

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

from ortools.linear_solver import pywraplp

# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
# Custom metrics
def precision_0_recall_1_inverse_weighted_fbeta(y_true, y_pred, beta=2.0):
    precisions, recalls, fbeta_scores, supports = precision_recall_fscore_support(y_true, y_pred, beta=beta, average=None)

    precision_0 = round(precisions[0], 4)
    recall_1 = round(recalls[1], 4)
    ratio_0, ratio_1 = supports / sum(supports)
    inverse_weighted_fbeta_score = round(fbeta_scores[0]*ratio_1 + fbeta_scores[1]*ratio_0, 4)
    
    return precision_0, recall_1, inverse_weighted_fbeta_score

In [3]:
# Loading data
def load_data(train_data_path, test_data_dir):
    # Training data
    train_data = np.load(train_data_path)
    
    # Testing data
    test_data_paths = glob(f"{test_data_dir}/*.npy")
    test_data_all = [np.load(test_data_path) for test_data_path in test_data_paths]
        
    return train_data, test_data_all

In [4]:
# Preprocessing data
def preprocess_data(train_data, test_data_all):
    # Training features and labels
    X_train = train_data[:, :-1]
    X_train = X_train / sum(X_train[0])
    y_train = train_data[1:, -1]

    # Testing features and labels
    X_test_all = [test_data[:, :-1] / sum(test_data[0, :-1]) for test_data in test_data_all]
    y_test_all = [test_data[1:, -1] for test_data in test_data_all]

    return X_train, y_train, X_test_all, y_test_all

In [5]:
def solve(train_data_path, test_data_dir):
    _, file_name = os.path.split(train_data_path)
    dist, num_days, _, num_samples, _, ratio = file_name.replace(".npy", "").split("_")

    # Load data
    train_data, test_data_all = load_data(train_data_path, test_data_dir)

    # Preprocess data
    X_train, y_train, X_test_all, y_test_all = preprocess_data(train_data, test_data_all)

    start_time = process_time()

    # Preparing interval frequencies
    min_edge, max_edge = 300, 850
    bin_edges = np.arange(min_edge, max_edge + 1, 1)

    train_size = len(bin_edges)
    num_days_train = X_train.shape[0]
    percent_days_train = np.zeros((num_days_train, train_size, train_size))

    for i in range(num_days_train):
        hist = X_train[i]
        for j in range(train_size - 1):
            for k in range(j + 1, train_size):
                percent_days_train[i, j, k] = np.sum(hist[j: k])    

    # Preparing PSIs
    epsilon = 1e-8 # Smoothing hyperparameters

    psi_train = []
    for i in range(1, num_days_train):
        psi_train.append((percent_days_train[i] - percent_days_train[i - 1]) * np.log((percent_days_train[i] + epsilon) / (percent_days_train[i - 1] + epsilon)))
    psi_train = np.array(psi_train)

    # PSI_0
    psi_0_train = psi_train[(1 - y_train).astype(bool)]
    psi_0_train = np.sum(psi_0_train, axis=0)
    # Normalization
    psi_0_train = psi_0_train / np.sum(1 - y_train)


    # PSI_1
    psi_1_train = psi_train[y_train.astype(bool)]
    psi_1_train = np.sum(psi_1_train, axis=0)
    # Normalization
    psi_1_train = psi_1_train / np.sum(y_train)


    end_time = process_time()
    preparing_data_time = end_time - start_time

    print(f"Time for preparing data: {preparing_data_time} s")

    # Declare the model
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Create the variables
    x = np.empty(shape=(train_size, train_size), dtype=object)

    for i in range(train_size):
        for j in range(train_size):
            if j > i:
                x[i, j] = solver.IntVar(0, 1, f'x[{i}, {j}]')
            else:
                x[i, j] = 0

    # Create the constraints
    start_time = process_time()

    # Each row/column has at most one 1
    # Non-overlap bins (a.k.a flow constraint)
    for i in range(1, train_size - 1):
        solver.Add(solver.Sum(x[: i, i]) <= 1)
        solver.Add(solver.Sum(x[i, i + 1:]) <= 1)
        solver.Add(solver.Sum(x[: i, i]) == solver.Sum(x[i, i + 1:]))
        
    # Ensure in-and-out
    solver.Add(solver.Sum(x[0, 1:]) == 1)
    solver.Add(solver.Sum(x[0: -1, -1]) == 1)

    # Ensure at most k bins
    max_num_bins = 25
    min_num_bins = 5
    solver.Add(solver.Sum(x.flatten()) <= max_num_bins)
    solver.Add(solver.Sum(x.flatten()) >= min_num_bins)

    end_time = process_time()
    constraints_time = end_time - start_time

    print(f"Time for creating constraints: {constraints_time} s")

    # Solve
    # Array fir storing results
    results = []

    # alphas = np.arange(0, 1.05, 0.05)
    # alphas = [round(alpha, 2) for alpha in alphas]
    alphas = [0.5]

    for alpha in alphas:  
        ########################
        ### current solution ###
        ########################
        result = [dist, num_days, num_samples, ratio, alpha, preparing_data_time, constraints_time]
        print(f"alpha = {alpha}")

        
        #######################
        ### Multi-objective ###
        #######################
        solver.Maximize(solver.Sum((alpha * psi_1_train * x).flatten()) - solver.Sum(((1 - alpha) * psi_0_train * x).flatten()))
        
        
        #########################
        ### Invoke the solver ###
        #########################
        start_time = process_time()
        status = solver.Solve()
        end_time = process_time()
        solving_time = end_time - start_time
        
        result.append(solving_time)
        print(f"Time for solving: {solving_time} s")
        
        
        ##########################
        ### Print the solution ###
        ##########################
        x_solution_value = np.zeros((train_size, train_size))

        for i in range(train_size):
            for j in range(train_size):
                if j > i:
                    x_solution_value[i, j] = x[i, j].solution_value()
                    
        final_bin_edges = []

        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            
            objective_0 = np.sum(psi_0_train * x_solution_value)
            result.append(objective_0)
            print(f"Objective_0 = {objective_0}")
            
            objective_1 = np.sum(psi_1_train * x_solution_value)
            result.append(objective_1)
            print(f"Objective_1 = {objective_1}")

            total_cost = solver.Objective().Value()
            result.append(total_cost)
            print(f"Total cost = {total_cost}", "\n")

            for i in range(train_size):
                for j in range(train_size):
                    if j > i and x[i, j].solution_value() == 1:
                        final_bin_edges.append(i + 300)
            final_bin_edges.append(max_edge)
        else:
            print('No solution found.')
            
        final_num_bin = len(final_bin_edges) - 1
        print("final_bin_edges =", final_bin_edges, "\n")
        result.append(final_num_bin)
                
        
        ###############
        ### Evaluation ###
        ###############
        # thresholds = np.arange(0.01, 1.01, 0.01)
        # thresholds = [round(threshold, 2) for threshold in thresholds]
        thresholds = [0.1]
                
        # Training Acccuracy & F1 & F0.5
        num_days_train = X_train.shape[0]
        best_train_threshold = best_train_precision_0 = best_train_recall_1 = best_train_inverse_weighted_f2 = 0
        best_y_train_pred = [0] * (num_days_train - 1)
        train_acc = 0
        
        for threshold in thresholds:
            y_train_pred = []
            
            for i in range(num_days_train - 1):
                hist_1 = []
                for j in range(len(final_bin_edges) - 1):
                    hist_1.append(np.sum(X_train[i, final_bin_edges[j] - 300: final_bin_edges[j + 1] - 300]))
                hist_1 = np.array(hist_1)

                hist_2 = []
                for j in range(len(final_bin_edges) - 1):
                    hist_2.append(np.sum(X_train[i + 1, final_bin_edges[j] - 300: final_bin_edges[j + 1] - 300]))
                hist_2 = np.array(hist_2)

                psis = (hist_1 - hist_2) * np.log((hist_1 + epsilon) / (hist_2 + epsilon))
                psi = np.sum(psis)
        
                if (y_train[i] == 0 and psi < threshold) or (y_train[i] == 1 and psi >= threshold):
                    y_train_pred.append(y_train[i])
                else:
                    y_train_pred.append(1 - y_train[i])
            
            train_precision_0, train_recall_1, train_inverse_weighted_f2 = precision_0_recall_1_inverse_weighted_fbeta(y_train, y_train_pred, beta=2.0)
            if train_inverse_weighted_f2 > best_train_inverse_weighted_f2:
                best_train_inverse_weighted_f2 = train_inverse_weighted_f2
                best_train_threshold = threshold
                best_train_precision_0 = train_precision_0
                best_train_recall_1 = train_recall_1
                best_y_train_pred = y_train_pred
                train_acc = accuracy_score(y_train, y_train_pred)

        print("Best threshold:", best_train_threshold)
        result.append(best_train_threshold)

        print("Training Accuracy:", train_acc)
        result.append(train_acc)

        print("Best Training Precision 0:", best_train_precision_0)
        result.append(best_train_precision_0)   

        print("Best Training Recall 1:", best_train_recall_1)
        result.append(best_train_recall_1)

        print("Best Training Inverse Weighted F2", best_train_inverse_weighted_f2)
        result.append(best_train_inverse_weighted_f2) 

        print(confusion_matrix(y_train, best_y_train_pred))
                
        # Testing Acccuracy & F1 & F2
        for i in range(len(X_test_all)):
            X_test, y_test = X_test_all[i], y_test_all[i]
            num_days_test = X_test.shape[0]
            y_test_pred = []
            psi_0_test = psi_1_test = 0

            for i in range(num_days_test - 1):
                hist_1 = []
                for j in range(len(final_bin_edges) - 1):
                    hist_1.append(np.sum(X_test[i, final_bin_edges[j] - 300: final_bin_edges[j + 1] - 300]))
                hist_1 = np.array(hist_1)

                hist_2 = []
                for j in range(len(final_bin_edges) - 1):
                    hist_2.append(np.sum(X_test[i + 1, final_bin_edges[j] - 300: final_bin_edges[j + 1] - 300]))
                hist_2 = np.array(hist_2)

                psis = (hist_1 - hist_2) * np.log((hist_1 + epsilon) / (hist_2 + epsilon))
                psi = np.sum(psis)

                if y_test[i] == 0:
                    psi_0_test += psi
                else:
                    psi_1_test += psi

                if (y_test[i] == 0 and psi < best_train_threshold) or (y_test[i] == 1 and psi >= best_train_threshold):
                    y_test_pred.append(y_test[i])
                else:
                    y_test_pred.append(1 - y_test[i])

            psi_0_test = psi_0_test / np.sum(1 - y_test)
            print("Testing Objective_0:", psi_0_test)
            result.append(psi_0_test)

            psi_1_test = psi_1_test / np.sum(y_test)
            print("Testing Objective_1:", psi_1_test)
            result.append(psi_1_test)

            total_objective_test = alpha*psi_1_test - (1 - alpha)*psi_0_test
            print("Testing Total Objective:", total_objective_test)
            result.append(total_objective_test)

            test_acc = accuracy_score(y_test, y_test_pred)
            print("Testing Accuracy:", test_acc)
            result.append(test_acc)
            
            test_precision_0, test_recall_1, test_inverse_weighted_f2 = precision_0_recall_1_inverse_weighted_fbeta(y_test, y_test_pred, beta=2.0)

            print("Testing Precision 0:", test_precision_0)
            result.append(test_precision_0)   

            print("Testing Recall 1:", test_recall_1)
            result.append(test_recall_1)

            print("Testing Inverse Weighted F2:", test_inverse_weighted_f2)
            result.append(test_inverse_weighted_f2)

            print(confusion_matrix(y_test, y_test_pred))

        results.append(result)

    return results


In [6]:
# Save results
def save_results(results, test_data_dir):
    test_data_paths = glob(f"{test_data_dir}/*.npy")
    id2file = {}
    for i in range(len(test_data_paths)):
        test_file = os.path.split(test_data_paths[i])[1].replace(".npy", "")
        id2file[i] = test_file

    df_columns = ["distribution", "num_days", "num_samples", "ratio", "alpha", 
                "preparing_data_time", "creating_constraints_time", "solving_time", 
                "objective_0", "objective_1", "total_cost", "final_num_bin", "best_threshold",
                "training_acc", "training_precision_0", "training_recall_1", "training_inverse_weighted_f2"]

    for i in range(len(test_data_paths)):
        df_columns.append(f"{id2file[i]}_objective_0")
        df_columns.append(f"{id2file[i]}_objective_1")
        df_columns.append(f"{id2file[i]}_total_objective")
        df_columns.append(f"{id2file[i]}_acc")
        df_columns.append(f"{id2file[i]}_precision_0")
        df_columns.append(f"{id2file[i]}_recall_1")
        df_columns.append(f"{id2file[i]}_inverse_weighted_f2")

    results_df = pd.DataFrame(results, columns=df_columns)

    return results_df

In [7]:
def main(train_data_list, test_data_dir, save_dir):
    items = [(train_data_path, test_data_dir) for train_data_path in train_data_list]
    total_results_df = pd.DataFrame()

    with Pool() as pool:
        for results in pool.starmap(solve, items):
            results_df = save_results(results, test_data_dir)
            total_results_df = pd.concat([total_results_df, results_df])

        total_results_df.to_csv(f"{save_dir}/test_results.csv", index=False)

In [8]:
train_data_list = ["../data/small_psi_data/logistic/logistic_30_days_1000_samples_70.npy"]
test_data_dir = "../data/test/logistic/plain"
save_dir = f"../output/test"

if __name__ == "__main__":
    main(train_data_list, test_data_dir, save_dir)

Time for preparing data: 61.021344375999995 s
Time for creating constraints: 7.278982006999996 s
alpha = 0.5
Time for solving: 860.4780620370001 s
Objective_0 = 0.020988372434621056
Objective_1 = 0.3858336783104368
Total cost = 0.18242265293790788 

final_bin_edges = [300, 500, 516, 520, 529, 530, 536, 541, 542, 546, 547, 552, 557, 558, 560, 565, 569, 610, 611, 653, 682, 757, 763, 766, 789, 850] 

Best threshold: 0.1
Training Accuracy: 1.0
Best Training Precision 0: 1.0
Best Training Recall 1: 1.0
Best Training Inverse Weighted F2 1.0
[[22  0]
 [ 0  7]]
Testing Objective_0: 0.02098837243462106
Testing Objective_1: 0.38583367831043686
Testing Total Objective: 0.1824226529379079
Testing Accuracy: 1.0
Testing Precision 0: 1.0
Testing Recall 1: 1.0
Testing Inverse Weighted F2: 1.0
[[22  0]
 [ 0  7]]
